In [ ]:
#### This notebook is dedicated to determining which users are pain patients


### ToDo

* Isolate people who have only texted about migraine vs. crps (etc). Look for systematic differences
* Do analysis on all words in texts
* Try doing the analysis on twitter users rather than tweets
* Combine terms like 'fibro' and 'fibromyalgia'
* Store lists of things to ignore/combine/categorize etc in spreadsheet and import?

### Notes
SQL query for counting tweets in which hashtag used:
SELECT h.hashtag, count(t.tweetID) AS totTweets FROM hashtags h INNER JOIN tweetsXtags t ON h.tagID = t.tagID GROUP BY hashtag ORDER BY totTweets DESC ;

#### This cell was imported from twitter_data_analysis 11/9/16
That was not the last time this notebook was updated 

# Setup
## Base imports

In [ ]:
#standard lib
import os
import string

#other people's property
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
from pandas import DataFrame, Series
import sqlalchemy

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
#Base Classes stuff
from FileSystemTools import *
from UtilityDecorators import *
from UtilityFunctions import *


BASE = getSystemRoot()
USER_PROFILE_TERM_FREQ_FILEPATH = '%s/Desktop/TEMPORARY/user-profile-word-counts.csv' % BASE
MIN_100_NO_STOPS = '/Users/adam/Desktop/TEMPORARY/user-profile-word-counts-min-100-ex-stops.csv'
MAP_PICKLE_PATH = '%s/Desktop/TEMPORARY/user-id-map' % BASE

In [ ]:
# Global variables

# The minimum frequency above which to
# exclude terms
MIN_FREQ = 100

## Create cursor for user data

In [ ]:
%cd twitteranalysis
%run -i DataTools/Cursors
# from DataTools.Cursors import UserCursor
u = UserCursor(language='en')

In [ ]:
user = u.next()
user

In [ ]:
%cd twitteranalysis
%run -i DataTools/Cursors

u = WindowedUserCursor()

In [ ]:
%cd twitteranalysis
%run -i DataTools/TweetORM

def userIter():
    maxrq = 2
    firstid = None
    pk_attr = DataTools.TweetORM.Users.userID
    qry = u.dao.session.query(DataTools.TweetORM.Users)
    q = qry
#     for i in range(0, 2):
#     while True:
    rec = None
    if firstid is not None:
        q = qry.filter(DataTools.TweetORM.Users.userID > firstid)
        
        for rec in q.order_by(DataTools.TweetORM.Users.userID).limit(maxrq):
            yield rec
        if rec is None:
            raise StopIteration
        # after iteration is complete
        # set the start id to the last retrieved record's id
    firstid = pk_attr.__get__(rec, pk_attr) if rec else None


In [ ]:
maxid = 531

In [ ]:
class J:
    def __init__(self, dao):
        self.firstId = 0
        self.limit = 4
        self.pk_attr = DataTools.TweetORM.Users.userID
        self.qry = dao.session.query(DataTools.TweetORM.Users)
        self.my_iter = self.a()
    
    def next(self):
        return next(self.my_iter)
        
    def a(self):
        
        while True:
            q = qry
            if self.firstId is not None:
                # get records with ids higher than our highest current
                q = qry.filter(self.pk_attr > self.firstId)
            rec = None
            for rec in q.order_by(self.pk_attr).limit(self.limit):
                yield rec
            if rec is None:
                break
            self.firstId = self.pk_attr.__get__(rec, self.pk_attr) if rec else None

In [ ]:
j = J(u.dao)

In [ ]:
uss = u.next()
uss.userID

In [ ]:
n = j.itt()

In [ ]:
next(n)

In [ ]:
itt.maxid

In [ ]:
ui = userIter()

In [ ]:
next(ui)

In [ ]:
q = q.filter(DataTools.TweetORM.Users.userID > 15)


In [ ]:
for rec in q.order_by(pk_attr).limit(2):
    firstid = pk_attr.__get__(rec, pk_attr) if rec else None
    print(firstid)
     

In [ ]:
# Reset database
%cd twitteranalysis
from DataTools.WordORM import create_db_tables


In [ ]:
create_db_tables()

# Windowing user cursor

In [ ]:
next(u.item_iterator)

In [ ]:
j = u.next()

In [ ]:
j

# Word counts in user descriptions

In [ ]:
%cd texttools
from TextProcessors.Filters import *
from TextProcessors.Modifiers import *
from TextProcessors.Processors import *

filters = [
     UsernameFilter(),
    PunctuationFilter(),
    URLFilter(),
    NumeralFilter()
]

modifiers = [
    WierdBPrefixConverter(), 
    CaseConverter( ) 
]

In [ ]:
%cd texttools
%run -i StatisticalTools/Counters
counter = WordCounter()
counter.add_filters(filters)
counter.add_modifiers(modifiers)

# Integrating with older tools

In [ ]:
# Statistical tools approach is better, but
# older version separates work better...

%cd twitteranalysis
%run -i ProcessingTools/QueueTools.py
%run -i ProcessingTools/ProcessingControllers.py
%run -i ProcessingTools/Listeners.py

# First set up the object which will handle applying
# filters and modifiers to each word
word_processor = SingleWordProcessor()
word_processor.add_filters(filters)
word_processor.add_modifiers(modifiers)

# Set up the machinery for saving the 
# processed results
queueHandler = SaveQueueHandler()
listener = SaveListener()
queueHandler.register_listener(listener)

# Finally create the command and control
control = UserProcessingController(queueHandler)
control.load_word_processor(word_processor)

In [ ]:
%cd twitteranalysis

# Make a fake user
%run -i TestingTools/Factories.py
%run -i TestingTools/DummyCursors.py
dummyCursor =DummyUserCursor()
# user = UserFactory()

In [ ]:
Faker().word()

In [ ]:
t =TweetResultFactory()
t.word_index

In [ ]:
import random
random.randint(0, 10)

In [ ]:
u = dummyCursor.next()
u.description

In [ ]:
def _yield_limit(qry, pk_attr, maxrq=100):
    """specialized windowed query generator (using LIMIT/OFFSET)

    This recipe is to select through a large number of rows thats too
    large to fetch at once. The technique depends on the primary key
    of the FROM clause being an integer value, and selects items
    using LIMIT."""

    firstid = None
    while True:
        q = qry
        if firstid is not None:
            q = qry.filter(pk_attr > firstid)
        rec = None
        for rec in q.order_by(pk_attr).limit(maxrq):
            yield rec
        if rec is None:
            break
        firstid = pk_attr.__get__(rec, pk_attr) if rec else None

from sqlalchemy import create_engine, Column, Integer
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class A(Base):
    __tablename__ = 'a'

    id = Column(Integer, primary_key=True)

e = create_engine("sqlite://", echo=True)
Base.metadata.create_all(e)

sess = Session(e)

sess.add_all([A() for i in range(2000)])

for rec in _yield_limit(sess.query(A), A.id):
    print(rec.id, rec)

# Streamlining db

In [ ]:
DB_HOST = '127.0.0.1'
DB_NAME = 'twitter_wordsTEST'
DB_USER = 'root'
DB_PASS = ''

In [ ]:
import MySQLdb as msq
import MySQLdb.cursors


class NonAutocommittingBaseDAO:
    """
    Base data access object. 
    Inherited by all classes that need to access the mysql database
    """

    def __init__( self, batchSize=100 ):
        self.mysqlError = MySQLdb.Error
        self.batchSize= batchSize
        self.cnt = 0
        
        try:

            self.db = msq.connect( DB_HOST, DB_USER, DB_PASS, DB_NAME,
                                   cursorclass=MySQLdb.cursors.DictCursor )
            self.db.autocommit( False )
            self.dbc = self.db.cursor()
            print( 'connected to: ', DB_NAME )
        except MySQLdb.Error as e:
            print( "Connection error: %s " % e )

    def check_flush(self):
        if self.batchSize <= self.cnt:
            self.db.commit()
            self.cnt = 0
            print('committed', self.cnt)
        

    def executeQuery( self, query, val ):
        """
        Prepares and executes the query stored in self.query with the variables in self.val
        Usually used for insert, update, and other functions which don't require a return
        """
        if type(val) is not tuple:
            val = (val,)
            
        try:
            self.dbc.execute( query, val )
            self.cnt += 1
            self.check_flush()

        except MySQLdb.Error as e:
            self.handleError(e)


    def returnOne( self, query, val ):
        """
        Executes the query stored in self.query with the vals in self.val.
        Returns the first row in an array called self.results
        """
        try:
            self.dbc.execute( query, val )
            self.results = self.dbc.fetchone()
        except MySQLdb.Error as e:
            self.handleError(e)

    def handleError(self, error):
        print( "Query failed: %s " % error )


    def returnAll( self, query, val ):
        """
        Executes the query stored in self.query with the vals in self.val.
        Return the results in an array called self.results
        """
        try:
            self.dbc.execute( query, val )
            self.results = self.dbc.fetchall()
        except MySQLdb.Error as e:
            self.handleError(e)
        

class WordDAO(NonAutocommittingBaseDAO):
    def __init__(self, batchsize=100):
        super().__init__(batchsize)
    
    def get_word_id(self, word):
        """Returns the id of the word if it already exists"""
        try:
            self.executeQuery("""SELECT id FROM words WHERE word = %s""", word)
            r = self.dbc.fetchone()
            return r['id']
        except:
            return self.add_word(word)

    def add_word(self, word):
        """
        Adds a new word to the words table
        Will return the id of the newly inserted word
        """
        q = """INSERT INTO words (word) VALUES (%s)"""
        self.executeQuery(q, word)
        self.db.commit()
        self.dbc.execute("""SELECT LAST_INSERT_ID() AS id""")
        r = self.dbc.fetchone()
        return r['id']

class WordMapDAO(NonAutocommittingBaseDAO):
    
    def __init__(self, batchsize=100):
        super().__init__(batchsize)

    
    def add(self, wordId, sentenceIndex, wordIndex, tweetId=None, userId=None):
        tweetQuery = """INSERT INTO word_map (word_id, sentence_index, word_index, tweet_id) 
            VALUES (%s, %s, %s, %s)"""
        
        userQuery = """INSERT INTO word_map (word_id, sentence_index, word_index, user_id) 
            VALUES (%s, %s, %s, %s)"""
        
        if tweetId is not None: 
            q = tweetQuery
            oid = tweetId
            
        if userId is not None:
            q = userQuery
            oid = userId
            
        return self.executeQuery(q, (wordId, sentenceIndex, wordIndex, oid))
        


In [ ]:
wd = WordDAO()

In [ ]:
wd.get_word_id('kjhkhkhkhhhhhh23saa')

In [ ]:
k = wd.add_word('faljer')
k

In [ ]:
wd.add_word('faljer2')

In [ ]:
d.db.commit()

# Working on async queuing 

In [ ]:
import asyncio
import random
from time import sleep
import functools

In [ ]:

@asyncio.coroutine 
def foo():
    print('Running in foo')
    yield from asyncio.sleep(0)
    print('Explicit context switch to foo again')

@asyncio.coroutine 
def bar():
    print('Explicit context to bar')
    yield from asyncio.sleep(0)
    print('Implicit context switch back to bar')


ioloop = asyncio.get_event_loop()
tasks = [ioloop.create_task(foo()), ioloop.create_task(bar())]
wait_tasks = asyncio.wait(tasks)
ioloop.run_until_complete(wait_tasks)
ioloop.close()

In [ ]:
def task(pid):
    """Synchronous non-deterministic task.
    """
    sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)

@asyncio.coroutine
def task_coro(pid):
    """Coroutine non-deterministic task
    """
    yield from asyncio.sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)


In [ ]:
ioloop = asyncio.get_event_loop()
result_queue = []

def enqueue(result):
    result_queue.append(ioloop.create_task(save_to_db(result)))

def handle_queue():
    wait_tasks = asyncio.wait(result_queue)
    ioloop.run_until_complete(wait_tasks)
    
@asyncio.coroutine 
def save_to_db(result):
    print("I'm saving %s to db" % result)
    

for i in range(0, 10):
    enqueue(i)
    handle_queue()
    if i == 10: ioloop.close()


    

#     loop = asyncio.get_event_loop()
#     try:
#         loop.call_soon(functools.partial






Here are the most basic definitions of asyncio main concepts:

Coroutine 

— generator that consumes data, but doesn’t generate it. Python 2.5 introduced a new syntax that made it possible to send a value to a generator. I recommend checking David Beazley’s “A Curious Course on Coroutines and Concurrency” for a detailed description of coroutines.
    
    
Tasks 

— schedulers for coroutines. If you check a source code below, you’ll see that it just says event_loop to run its _step as soon as possible, meanwhile _step just calls next step of coroutine.


In [ ]:

import asyncio  
import time  
from datetime import datetime

class Task(asyncio.futures.Future):  
    def __init__(self, coro, loop=None):
        super().__init__(loop=loop)
#         ...
        self._loop.call_soon(self._step)

    def _step(self):
#             ...
        try:
#             ...
            result = next(self._coro)
        except StopIteration as exc:
            self.set_result(exc.value)
        except BaseException as exc:
            self.set_exception(exc)
            raise
        else:
#             ...
            self._loop.call_soon(self._step)

@asyncio.coroutine
def custom_sleep():  
    print('SLEEP', datetime.now())
    yield from asyncio.sleep(1)
#     time.sleep(1)

@asyncio.coroutine
def factorial(name, number):  
    f = 1
    for i in range(2, number+1):
        print('Task {}: Compute factorial({})'.format(name, i))
        yield from custom_sleep()
        f *= i
    print('Task {}: factorial({}) is {}\n'.format(name, number, f))

start = time.time()  
loop = asyncio.get_event_loop()

tasks = [  
    factorial("A", 3),
    factorial("B", 4),
]
loop.run_until_complete(asyncio.wait(tasks))  
loop.close()

end = time.time()  
print("Total time: {}".format(end - start))

In [ ]:



import asyncio

@asyncio.coroutine
def my_task(seconds):
    """
    A task to do for a number of seconds
    """
    print('This task is taking {} seconds to complete'.format(seconds))
    yield from asyncio.sleep(seconds)
    return 'task finished'
 
def run():
    my_event_loop = asyncio.get_event_loop()
    try:
        print('task creation started')
        task_obj = my_event_loop.create_task(my_task(seconds=2))
        my_event_loop.run_until_complete(task_obj)
    finally:
        my_event_loop.close()
        print('done')



In [ ]:
run()


In [ ]:
from itertools import izip

def finish_save(response, result):
    if response.code != 200:
        print "Error saving prime: {}".format(prime)

def do_thing(t):
    pass

class AsyncBatcher(object):
    __slots__ = ["batch", "batch_size", "save", "flush"] def __init__(self, batch_size):
    self.batch_size = batch_size
    self.batch = []

    def save(self, result):
        url = "bzzzzrrrrrppp saaaaavvvvviiiinnngggg"
        self.batch.append((url,result))
        if len(self.batch) == self.batch_size:
            self.flush()

def flush(self):
    responses_futures = (do_thing(url) for url, _ in self.batch) 
    responses = grequests.map(responses_futures)
    for response, (url, result) in izip(responses, self.batch):
        finish_save(response, result)
    self.batch = []

# Tornado 

In [ ]:
# send result to db server to be recorded

%cd twitteranalysis
import environment
url = environment.DB_URL

%run -i Servers/Helpers.py
%run -i Servers/ClientSide.py
%run -i TestingTools/Factories.py

# from tornado.httpclient import AsyncHTTPClient


In [ ]:
# payload = UserResultFactory()
        
# http_client = AsyncHTTPClient() 

# send_result(http_client, url, payload)

# http_client.close()

In [ ]:
# tornado logging
access_log = logging.getLogger("tornado.access")
app_log = logging.getLogger("tornado.application")
gen_log = logging.getLogger("tornado.general")

In [ ]:
# Create new log
%cd ~/Desktop/TwitterDataAnalysisLogs
%sx rm query_time_log.csv
%sx touch query_time_log.csv
%sx rm query_log.csv
%sx touch query_log.csv
%sx rm wordmapping.db
%sx touch wordmapping.db

To audit:
Before running, execute in db: 
    
    DELETE FROM word_map; DELETE FROM words

After run, compare count of users to 

SELECT count( DISTINCT user_id) FROM word_map;

SELECT count( DISTINCT userID) FROM users;
    
332 have blank profiles

In [ ]:
# check stored sqlite file
%cd twitteranalysis
import sqlite3
import environment

f = environment.SQLITE_FILE

conn = sqlite3.connect(f)
cursor = conn.cursor()

In [ ]:
r = cursor.execute('SELECT count(word) FROM word_map_deux')

In [ ]:
for i in r:
    print(i)

SQlite 4.17.18
4352 users processed (not nec done)

67943 responses

4352 profiles (of 4353) took 15.318603992462158 seconds. That's 0.00351989981444443 seconds per profile

db done after 3min 31 sec; recorded 43784 (1.1mb)

mysql 4.17
4352 users processed (not nec done)
67943 responses
4352 profiles (of 4353) took 15.312575101852417 seconds. That's 0.00351851449950653 seconds per profile

db done after ~4 min (3.5mb); recorded 67943

In [ ]:
numberUsersProcessed = 4352
totalUsers = 1328927

secPerUser = numberUsersProcessed / (4*60)
estSec = totalUsers * secPerUser
print("estimated time: %s seconds" % estSec)

In [ ]:
estHours = (estSec / 60) / 60
estHours

In [ ]:
%cd twitteranalysis

from DataTools import DataConnections
from DataTools import WordORM
ENGINE = 'sqlite'
            
engine = DataConnections.initialize_engine()
WordORM.create_db_tables(engine)
# DataTools's handle to database at global level

      

In [ ]:
engine.execute("select * from words")

In [ ]:
len(control.responses)

In [ ]:
control.prune_responses()

In [ ]:
payload = UserResultFactory()

client = Client()
f = client.send(payload)
f.done()
# client.close()

In [ ]:
f.done()

In [ ]:
client.http_client

In [ ]:
DB_URL

### Redis as word id lookup

In [ ]:
import redis


class RedisDAO(object):
    """
    Creates connection to redis host. 
    Base class for redis services. Should not be directly called.

    Attributes:
        dbindex: Default 'db0'
        db: redis.Redis connection
    """

    def __init__(self, host='localhost', port=6379, db=0):
        d = str(db)
        #self.dbindex = 0
        self.dbindex = 'db%s' % d
        self.db = redis.Redis(host=host, port=port, db=db)

    def delete_all_databases(self):
        self.flushall()
#         pass





In [ ]:

    def save_tweet_id_to_master_list(self, tweetid):
        self.setname = 'tweetIDs'
        try:
            tid = 'unique_tweets:tweet_id:%s' % tweetid
            self.db.sadd(tid, tid)
        except Exception as e:
            print "error %s" % e

    def save_tweet_id(self, tweetid):
        """
        @param tweetid This is the id to be recorded in redis
        @type tweetid Number or string
        """
        # Name of the set that will be storing all the data
        self.setname = 'tweetIDs'
        try:
            #tid = 'unique_tweets:tweet_id:%s' % tweetid
            self.db.sadd(self.setname, tweetid)
        except Exception as e:
            print "error %s" % e

    def get_max_id(self):
        try:
            self.maxid = self.db.sort('tweetIDs', start=0, num=1, desc=True)
            return self.maxid
            # self.loadkeys()
            # mx = max(self.keys)
            # self.maxid = RedisFormat.tweet_id_remove(mx)
        except Exception as e:
            print "Error: %s " % e



In [ ]:
Us

In [ ]:
user = u.next()
user.description

In [ ]:
control.process(user)

In [ ]:
queueHandler.queue.queue

In [ ]:
# This model works faster
us = []
for i in range(0, 20):
    us.append(u.next())

[ control.process(u) for u in us]

In [ ]:
# Next task: getting the queueHandler to do its job
# and save to sql

### Process the user descriptions for word counts

In [ ]:
# Don't know why but this is incredibly slow
# restricted number of users for testing
# max_users = 2

# for i in range(0, max_users):
#     user = u.next()
#     upc.process(user)
# #     counter.process(user.description)


In [ ]:
sq.queue.queue

In [ ]:

def process_word_freq_in_user_profiles(counter, userCursor):
    # all users
    user_count = 0

    while True:
        try:
            user = userCursor.next()
    #         Note that we're not going to add the id to the map yet
            counter.process(user.description)
            user_count += 1
        except StopIteration as e:
            print("%s users processed; %s words identified" % (user_count, len(counter.counts)))
            break

    return Series(counter.counts)

# word_freq_in_user_profiles = process_word_freq_in_user_profiles(counter, u)
# word_freq_in_user_profiles.to_csv(USER_PROFILE_TERM_FREQ_FILEPATH)

# took 6m31s to process 
# 1162362 users processed; 846948 words identified


### Report word frequencies in user profiles

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
word_freq_in_user_profiles.hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title("Term frequency in user profiles")
fig.tight_layout()

## Truncate the found word list

In [ ]:
# Remove terms 
words = word_freq_in_user_profiles.loc[lambda x: x > MIN_FREQ]
print("%s words out of %s appear more often than %sx" % (len(words), len(word_freq_in_user_profiles), MIN_FREQ))

In [ ]:
out = '/Users/adam/Desktop/TEMPORARY/user-profile-word-counts-over-100.csv'
# words.to_csv(out)

In [ ]:
fig, ax = plt.subplots()
words.hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title("Term frequency in user profiles (min: %s)" % MIN_FREQ)
fig.tight_layout()

## Remove stopwords 

In [ ]:
def is_stopword(text):
    """check whether the supplied text is a stopword"""
    return text in nltk.corpus.stopwords.words('english')

def remove_stopwords(series):
    d = {}
    for k in series.keys():
        if not is_stopword(k):
             d[k] = series[k]
    print("Removed %s stopwords from the words list" % (len(series) - len(d.keys())))
    return Series(d)

words = remove_stopwords(words)
#words.to_csv(MIN_100_NO_STOPS)

### Word frequencies in user profiles

In [ ]:
words = pd.read_csv(MIN_100_NO_STOPS, header=None, names=['term', 'freq'])
len(words)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
words.freq.hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title("User profile term freq (min: %s ; stopwords removed)" % MIN_FREQ)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
sns.distplot(words.freq, ax=ax)
ax.set_title("KDE of Term frequency in user profiles")

In [ ]:
words.max()

In [ ]:
w = words.sort_values(by='freq', ascending=False)
w[:10]

In [ ]:
j = pd.read_csv(MIN_100_NO_STOPS)

In [ ]:
len(j)

In [ ]:
ww = words.copy(deep=True)[:10]
w5 = ww.loc[lambda x : x.freq > 1000]
w5

In [ ]:
j[:5]

In [ ]:
# Remove terms 
def filter_freq_list_by_min(wordFrame, minFreq):
    """Returns a copy of the frame, sans terms which do not occur above the minimum frequency"""
    w = wordFrame.loc[lambda x: x.freq > minFreq]
    print("%s words out of %s appear more often than %sx" % (len(w), len(wordFrame), minFreq))
    return w


In [ ]:
w4 = filter_freq_list_by_min(words, 10000)
len(w4)

In [ ]:
w4

In [ ]:
%cd texttools
%run -i StatisticalTools/Counters
selectedCounter = SelectedWordCounter()
selectedCounter.set_words_to_count(list(counter.map.keys()))

In [ ]:
selectedCounter.map

In [ ]:
# restricted number of users for testing
max_users = 10



for i in range(0, max_users):
    user = u.next()
    selectedCounter.process(user.description, user.userID)

    
# todo save data

In [ ]:
selectedCounter.counts

In [ ]:
usersWithTermInProfile =[]

In [ ]:

def get_rows_for_terms(wordFrame, experimentalTerms):
    return wordFrame[wordFrame.term.isin(experimentalTerms)]

EXP_TERMS_FILEPATH = '%s/Dropbox/PainNarrativesLab/Data/experimental-terms.xlsx' % BASE
experimentalTerms = pd.read_excel(EXP_TERMS_FILEPATH, squeeze=True)

In [ ]:
get_rows_for_terms(words, experimentalTerms)

# Logger

In [ ]:
from FileSystemTools import *
from UtilityDecorators import *
from UtilityFunctions import *


BASE = getSystemRoot()
LOG_FOLDER_PATH = "%s/Desktop/TwitterDataAnalysisLogs" %BASE
DEFAULT_LOG_FILE_NAME = 'twitter_log.txt'
DEFAULT_LOG_FILE_PATH = "%s/%s" % (LOG_FOLDER_PATH, DEFAULT_LOG_FILE_NAME)


In [ ]:
%cd tweetloggers
%run -i FileLoggers

In [ ]:
%bookmark 

In [ ]:
f = FileWritingLogger(name='best')

In [ ]:
f.log('j')

In [ ]:
fwl = FileWritingLogger()

In [ ]:
fwl.log('taco')

In [ ]:
fwl2 = FileWritingLogger('new')

In [ ]:
f.log_error('tacoooooo')

In [ ]:
sol = StdOutLogger()

In [ ]:
sol.log('taco')

# Categorize users by condition

In [ ]:
# Determine number of valid users identified and number of tweets each

In [ ]:
stripNonAlphaNum('!jip!')

In [ ]:

wordTokenizer = WordTokenizer()
word_processor = SingleWordProcessor()


word_processor.add_to_filters( TextProcessors.Filters.UsernameFilter( ) )
word_processor.add_to_filters( TextProcessors.Filters.PunctuationFilter( ) )
word_processor.add_to_filters( TextProcessors.Filters.URLFilter( ) )
word_processor.add_to_filters( TextProcessors.Filters.NumeralFilter( ) )
word_processor.add_to_modifiers( TextProcessors.Modifiers.WierdBPrefixConverter() )
# processor.add_to_modifiers( TextProcessors.Modifiers.UnicodeConverter() )
word_processor.add_to_modifiers( TextProcessors.Modifiers.CaseConverter( ) )


# Create queue and listeners for processed tokens
Queue = SaveQueueHandler()
Queue.register_listener(SaveListener())

# Load cursor for tweet ids
cursor = DataTools.Cursors.TweetCursor()

StringProcessingWorker.initialize(cursor, Queue, word_processor)
threads = []

# for _ in range(1):
worker = StringProcessingWorker()
# worker.do_it()



In [ ]:
%cd textanalysis
%run -i ProcessingTools/SearchTools.py


In [ ]:
s = Searcher()
spoonie_results = s.search('Spoonie')

In [ ]:
# This cell was imported from twitter_data_analysis 11/9/16 
%cd textanalysis
%run -i ProcessingTools/SearchTools.py

# \section{Word frequency}
# freq = pd.read_excel("%s/freq_wordlist.xlsx" % DATAFOLDER)
# len(freq)

# fig, ax = plt.subplots()
# freq.freq.hist(ax=ax, bins=100, bottom=0.1)
# ax.set_yscale('log')
# ax.set_title("Term frequency in user profiles")

# trimmed_freq = freq[freq.freq > 1000]
# len(trimmed_freq)

# fig, ax = plt.subplots()
# trimmed_freq.freq.hist(ax=ax, bins=100, bottom=0.1)
# ax.set_yscale('log')
# ax.set_title("Term frequency in user profiles")

from beautifulsoup4.BeautifulSoup import BeautifulStoneSoup

def formatter(text):
    try:
        return unicode(BeautifulStoneSoup(text, convertEntities=BeautifulStoneSoup.ALL_ENTITIES))
    except:
        print ("Error %s" % text)
        return ''


trimmed_freq['word'] = trimmed_freq['word'].map(lambda x: formatter(x))

trimmed_freq.sort('freq', ascending=False)[:10]

fibro_aliases = ['Fibromyalgia', 'Fibro', 'fibro*']
for t in fibro_aliases:
    fibro = condition_searcher(t, fibro)

crps = Condition('crps')
crps_aliases = ['crps', 'RSD', 'c.r.p.s.', 'r.s.d.', 'complex regional pain syndrome', 'reflex sympathetic dystrophy']
for t in crps_aliases:
    crps = condition_searcher(t, crps)

import numpy
d = numpy.genfromtxt('/Users/adam/Desktop/freq_wordlist.txt')

import asciitable
h = DataFrame(asciitable.read('/Users/adam/Desktop/freq_wordlist.txt'))

dt = pd.read_table('/Users/adam/Desktop/freq_wordlist.csv', delim_whitespace=True, encoding='utf-8')

dt = dt.applymap(lambda x: x['word'].encode('ascii', 'replace'))

s = Searcher()
crps_results = s.search('((crps) | (RSD) | (r.s.d.) | (c.r.p.s.) | (complex regional pain syndrome) | (chronic regional pain syndrome) | (reflex sympathetic dystrophy))')
s = Searcher()
migraine_results = s.search('((migraine) | (Migraineur) | (migr*))')
s = Searcher()
fibro_results = s.search('((Fibromyalgia) | (Fibro) | (fibro*) | (fm) | (fms))')
s = Searcher()
spoonie_results = s.search('Spoonie')
s = Searcher()
vulvodynia_results = s.search('Vulvodynia | Vulvadynia')
s = Searcher()
endo_results = s.search('endometriosis | endo')
s = Searcher()
neuropathy_results = s.search('neuropathy')
s = Searcher()
arthritis_results = s.search('((arthritis) | (*arthritis) | (oa) | (ra))' )
s = Searcher()
neuralgia_results = s.search('(neuralgia) | (*neuralgia)')
s = Searcher()
shingles_results = s.search('((shingles) | (post-herpetic neuralgia) | (PHN))')
s = Searcher()
backpain_results = s.search('(back pain | backpain)')
s = Searcher()
headache = s.search('headache')

migraine_users = ug.get_from_list('migraine', migraine_results['userids'])
crps_users = ug.get_from_list('crps', crps_results['userids'])
fibro_users = ug.get_from_list('fibromyalgia', fibro_results['userids'])
spoonie_users = ug.get_from_list('spoonie', spoonie_results['userids'])
vulvodynia_users = ug.get_from_list('vulvodynia', vulvodynia_results['userids'])
endo_users = ug.get_from_list('endometriosis', endo_results['userids'])
neuropathy_users = ug.get_from_list('neuropathy', neuropathy_results['userids'])
arthritis_users = ug.get_from_list('arthritis', arthritis_results['userids'])
neuralgia_users = ug.get_from_list('neuralgia', neuralgia_results['userids'])
shingles_users = ug.get_from_list('shingles', shingles_results['userids'])
backpain_users = ug.get_from_list('backpain', backpain_results['userids'])

migraine_users.to_csv('%s/migraine.csv' % DATAFOLDER)
crps_users.to_csv('%s/crps.csv' % DATAFOLDER)
fibro_users.to_csv('%s/fibromyalgia.csv' % DATAFOLDER)
spoonie_users.to_csv('%s/spoonie.csv' % DATAFOLDER)
vulvodynia_users.to_csv('%s/vulvodynia.csv' % DATAFOLDER)
endo_users.to_csv('%s/endo.csv' % DATAFOLDER)
neuropathy_users.to_csv('%s/neuropathy.csv' % DATAFOLDER)
arthritis_users.to_csv('%s/arthritis.csv' % DATAFOLDER)
neuralgia_users.to_csv('%s/neuralgia.csv' % DATAFOLDER)
shingles_users.to_csv('%s/shingles.csv' % DATAFOLDER)
backpain_users.to_csv('%s/backpain.csv' % DATAFOLDER)

In [ ]:
j='jjj'
print("%s" % j)

In [ ]:
TextTools.TextFilters.remove_numerals("7")

In [ ]:
"""
Used to be in FILEFOLDER = '%s/Desktop/user_categories' % BASE
Moved into Data folder in pain narratives lab
"""
def file_getter(filefolder=FILEFOLDER):
    """
    Loads the the files in the folder    
    Args:
        filefolder: Folder path for where all the folders are stored
    
    Returns:
        Dataframe
    """
    datafiles = []
    sourceFolder = os.walk(filefolder)
    for f in sourceFolder:
        filelist = f[2]
        for fl in filelist:
            fl = str(fl)
            #if fl is not '.DS_Store':
            if fl[-5:] == '.xlsx':
                print (fl)
                loc = filefolder + '/' + str(fl)
                datafiles.append(loc)
    return datafiles


def cat_importer(filenames):
    """
    Import spreadsheets with categorizations of users, 
    combine them, and return the result
        
    Args:
        filenames: List of full path excel files
    
    Returns:
        Dataframe
    """
    frames = []
    for f in filenames:
        try:
            frames.append(pd.read_excel(f))
        except:
            print ("error with %s" % f)
    combined = pd.concat(frames)
    return combined

In [ ]:
files = file_getter()
compiled = cat_importer(files)
print (len(compiled))

In [ ]:
compiled.columns

In [ ]:
# Make frames with categorized users
compiled = compiled[compiled.relevant == 1]
patients = compiled[compiled.patient == 1]
clinicians = compiled[compiled.clinician == 1]

# Separate by condition
patients_by_condition = patients.groupby('term')
clinicians_by_condition = clinicians.groupby('term')

In [ ]:
for g, n in patients_by_condition:
    print(g, len(n)) 

In [ ]:
migraine_bag = []
m = patients_by_condition.get_group('migraine')
for i in m.profile:
    migraine_bag.append(word_tokenize(i))
#for p in m:
#    migraine_bag.append(word_tokenize(p.profile))
print(len(migraine_bag))

In [ ]:

bagmaker = TextTools.WordBagMaker()
bagmaker.add_to_ignorelist(ConstantsAndUtilities.Ignore.get_list())
bagmaker.add_to_ignorelist(list(string.punctuation))
bagmaker.add_to_ignorelist(nltk.corpus.stopwords.words('english'))
bagmaker.add_to_cleaners(TextTools.URLCleaner())
#bagmaker.add_to_cleaners(TextTools.NumeralCleaner())

In [ ]:
'3'.isalpha()

In [ ]:
nc = TextTools.NumeralCleaner()

In [ ]:
nc.clean('cat')

In [ ]:
m = patients_by_condition.get_group('migraine')
pw = [w for w in m.profile]

bagmaker.process(pw)
print(len(bagmaker.masterbag))

In [ ]:
bagmaker.masterbag

In [ ]:
wf = TextStats.WordFreq(bagmaker.masterbag)

In [ ]:
wf.topN(20)

In [ ]:
wf.plot(30)

In [ ]:
mb = tuple(migraine_bag)
mfd = nltk.FreqDist(mb)

In [ ]:
%cd twitteranalysis
import DataTools.Cursors

# Create tables in database
from DataTools.TweetORM import create_db_tables
# create_db_tables()

#%cd /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
#%bookmark twitteranalysis
%cd twitteranalysis
%run -i environment.py
%run -i ConstantsAndUtilities.py
%run -i TestingTools/DataAndFunctionsForTesting.py
%run -i "DataTools/DataStructures.py"
%run -i "DataTools/DataConnections.py"
%run -i "DataTools/WordORM.py"
%run -i "DataTools/DataRepositories.py" 

# Initialize the tools for filtering and modifying the individual tweet words
from TextProcessors.Filters import *
from TextProcessors.Modifiers import *
%cd texttools
%run -i TextProcessors/Processors
%run -i TextProcessors/Tokenizers

%run -i ProcessingTools/ProcessingControllers.py
%run -i ProcessingTools/Listeners.py
%run -i ProcessingTools/Workers.py
import DataTools.Cursors